In [180]:
import numpy as np
import itertools

In [181]:
fname = "GroceryStoreDataSet.csv"

In [182]:
def getSubsets(L, n): 
    s = []
    for i in itertools.combinations(L, n):
        s.append(frozenset(i))
    return s

In [183]:
def check(L, x):
    n = len(x) - 1
    s = getSubsets(x, n)
    for i in s:
        if (i not in L):
            return False
    return True

In [184]:
def candidate1(fname):
    f = open(fname, "rt")
    cList = {}
    nt = 0
    for line in f.readlines():
        nt += 1
        line = line.strip()
        line = line.strip('"')
        line = line.split(',')
        for item in line:
            i = cList.get(item, 0)
            if (i == 0):
                cList[item] = 1
                continue
            cList[item] += 1
    return cList, nt

In [185]:
def frequent(cList, nt, sup):
    fList = []
    for item in cList:
        if (cList[item]/nt >= sup):
              fList.append(item)
    return fList

In [186]:
def generateCandidate(L):
    cList = []
    n = len(L[0])
    m = len(L)
    for i in range(m):
        for j in range(i+1, m): 
            temp1 = list(L[i])[:n-1]
            temp2 = list(L[j])[:n-1] 
            temp1.sort() 
            temp2.sort()
            if (temp1 == temp2):
                new = (L[i] | L[j])
                new = frozenset(new)
                if (check(L, new)):
                    cList.append(new)
    return cList

In [187]:
def getCandidates(fname, L):
    LT = L[:]
    f = open(fname,"rt")
    cList = {}
    n = len(LT[0])
    for line in f.readlines():
        line = line.strip()
        line = line.replace('"','')
        temp = line.split(',')
        s = getSubsets(temp, n)
        for i in LT:
            if (i in s):
                count = cList.get(i,0)
                cList[i] = count+1
    return cList

In [188]:
def apriori(fname, sup = 0.4):
    freqSet = {}
    C1, n = candidate1(fname)
    L1 = frequent(C1, n, sup)
    for i in L1:
        freqSet[frozenset({i})] = C1[i]
    C2 = getSubsets(L1, 2)
    while len(C2) != 0:
        C = getCandidates(fname, C2)
        L2 = frequent(C, n, sup)
        if (len(L2) == 0):
            break
        for i in L2:
            freqSet[i] = C[i]
        C2 = generateCandidate(L2)
    return freqSet, n

In [189]:
def discoverRules(freqSet, confidence = 0.6):
    rules = {}
    conL = []
    supL = []
    for key in freqSet.keys():
        if len(key) == 1:
            continue
        n = len(key)
        for k in range(1, n):
            s = list(itertools.combinations(key, k))
            for i in s:
                if (freqSet[key]/freqSet[key-frozenset(i)]) >= confidence:
                    rules[key-frozenset(i)] = frozenset(i)
                    conL.append(freqSet[key]/freqSet[key-frozenset(i)])
                    supL.append(freqSet[key])
    return rules, conL, supL

In [190]:
freq, nt = apriori(fname, 0.15)

In [191]:
freq

{frozenset({'MILK'}): 5,
 frozenset({'BREAD'}): 13,
 frozenset({'BISCUIT'}): 7,
 frozenset({'CORNFLAKES'}): 6,
 frozenset({'TEA'}): 7,
 frozenset({'BOURNVITA'}): 4,
 frozenset({'MAGGI'}): 5,
 frozenset({'COFFEE'}): 8,
 frozenset({'CANDY'}): 3,
 frozenset({'SUGER'}): 6,
 frozenset({'BREAD', 'MILK'}): 4,
 frozenset({'BISCUIT', 'BREAD'}): 4,
 frozenset({'BISCUIT', 'CORNFLAKES'}): 3,
 frozenset({'BREAD', 'TEA'}): 4,
 frozenset({'BOURNVITA', 'BREAD'}): 3,
 frozenset({'BREAD', 'MAGGI'}): 3,
 frozenset({'MAGGI', 'TEA'}): 4,
 frozenset({'COFFEE', 'CORNFLAKES'}): 4,
 frozenset({'CANDY', 'COFFEE'}): 3,
 frozenset({'COFFEE', 'SUGER'}): 4,
 frozenset({'BREAD', 'COFFEE'}): 3,
 frozenset({'BREAD', 'SUGER'}): 4}

In [192]:
def runAlgorithm(fname):    
    freqSet, nt = apriori(fname, 0.15)
    rules, conL, supL = discoverRules(freqSet, 0.6)
    j = 0
    for i in rules:
        print(set(i), "-->", set(rules[i]), " with confidence = ", conL[j], " and support = ", supL[j]/nt)
        j += 1

In [193]:
runAlgorithm(fname)

{'MILK'} --> {'BREAD'}  with confidence =  0.8  and support =  0.2
{'BOURNVITA'} --> {'BREAD'}  with confidence =  0.75  and support =  0.15
{'MAGGI'} --> {'TEA'}  with confidence =  0.6  and support =  0.15
{'CORNFLAKES'} --> {'COFFEE'}  with confidence =  0.8  and support =  0.2
{'CANDY'} --> {'COFFEE'}  with confidence =  0.6666666666666666  and support =  0.2
{'SUGER'} --> {'BREAD'}  with confidence =  1.0  and support =  0.15
